In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import pandas as pd
import numpy as np
import pyodbc
from IPython.display import display
pd.options.display.max_colwidth = 1000

In [ ]:
nlp = spacy.load("en_core_web_lg")
nlp("J.J Moons").similarity(nlp("J.J Moons"))

In [ ]:
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
print(STOPWORDS)

In [ ]:
#Connect to the SQL Server to download Database 1 site 

In [ ]:
conx = pyodbc.connect("driver={SQL SERVER}; server=failoverjoblogiclivegroup.secondary.database.windows.net; #name_of_datbase; UID= #UserID_for_database; PWD= #database_password;")

In [ ]:
#write database into dataframe

In [ ]:
df_Site = pd.read_sql_query('SELECT * FROM reporting.Site', conx)
df_Site

In [ ]:
#write database 2 (Excel database) into dataframe

In [ ]:
df_Masterdata = pd.read_excel(r'#location of the database in your PC')
df_Masterdata

In [ ]:
#select the columns to be compared and rename to match the columns in SQL server.
df_Masterdata = df_Masterdata[["Outlet Id", "Outlet Name","Outlet Street" ,"Outlet Postcode"]]

In [ ]:
#Quick comparison to see how much site in database 2 exist in database 1

In [ ]:
df_Site['Site'].isin(df_Masterdata['Site']).value_counts()

In [ ]:
#merge both dataframes using unique site key

In [ ]:
#Convert CustomReference in Site to int32
df_Site=df_Site.dropna(subset=['CustomReference'])
df_Site2 = df_Site[pd.to_numeric(df_Site['CustomReference'], errors='coerce').notnull()]
df_Site2['CustomReference'] = df_Site2['CustomReference'].astype(int)
df_Site2.dtypes
df_Site2

In [ ]:
#Merge both dataframes using CustomReference
df_1=df_Site2.merge(df_Masterdata, on ="CustomReference", how='left')
df_1

In [ ]:
#Rename to makesure column names are clear
df_1 = df_1.rename(columns={"Site_x":"JL_Site","Site_y":"POC_Site","Address1_x":"JL_Address","Address1_y":"POC_Address","PostCode_x":"JL_PostCode","PostCode_y":"POC_PostCode"})
df_1

In [ ]:
#Convert types for columns to string
df_1['JL_Site'] = df_1['JL_Site'].astype(str)
df_1['POC_Site'] = df_1['POC_Site'].astype(str)
df_1['JL_Address'] = df_1['JL_Address'].astype(str)
df_1['POC_Address'] = df_1['POC_Address'].astype(str)

In [ ]:
#make value in column lowercase
df_1['JL_Site'] = df_1['JL_Site'].apply(lambda x:x.lower())
df_1['POC_Site'] = df_1['POC_Site'].apply(lambda y:y.lower())
df_1['JL_Address'] = df_1['JL_Address'].apply(lambda x:x.lower()) 
df_1['POC_Address'] = df_1['POC_Address'].apply(lambda y:y.lower()) 

In [ ]:
#remove stop words from site name
df_1['JL_Site'] = df_1['JL_Site'].astype(str).apply(lambda x: remove_stopwords(x))
df_1['POC_Site'] = df_1['POC_Site'].astype(str).apply(lambda x: remove_stopwords(x))
df_1

In [ ]:
#Identify Nouns Pronouns and Entity within site names
df_1['JL_SiteNoun'] =df_1['JL_Site'].apply(lambda x : " ".join([str(token) for token in nlp(x) if token.pos_ == "NOUN"]))
df_1['JL_SitePronoun'] =df_1['JL_Site'].apply(lambda x : " ".join([str(token) for token in nlp(x) if token.pos_ == "PROPN"]))
df_1['JL_SiteEntity'] = df_1['JL_Site'].apply(lambda x : " ".join([str(token) for token in nlp(x).ents]))
df_1['JL_POCNoun'] =df_1['POC_Site'].apply(lambda x : " ".join([str(token) for token in nlp(x) if token.pos_ == "NOUN"]))
df_1['JL_POCPronoun'] =df_1['POC_Site'].apply(lambda x : " ".join([str(token) for token in nlp(x) if token.pos_ == "PROPN"]))
df_1['JL_POCEntity'] = df_1['POC_Site'].apply(lambda x : " ".join([str(token) for token in nlp(x).ents]))

In [ ]:
#test to make sure the noun conversation is working. 
df_1['JL_Site'].iloc[0:10].apply(lambda x : " ".join([str(token) for token in nlp(x) if token.pos_ == "NOUN"]))

In [ ]:
#write to CSV so you don't have to run conversation everytime you run the code
df_1.to_csv('SiteNLP.csv')

In [ ]:
#read CSV back in
df_1 = pd.read_csv(r'#File location for the CSV file')
df_1.dtypes

In [ ]:
#Convert columns to string
df_1['JL_SiteNoun']= df_1['JL_SiteNoun'].astype(str)
df_1['JL_POCNoun']= df_1['JL_POCNoun'].astype(str)
df_1['JL_SitePronoun']= df_1['JL_SitePronoun'].astype(str)
df_1['JL_POCPronoun']= df_1['JL_POCPronoun'].astype(str)
df_1['JL_SiteEntity']= df_1['JL_SiteEntity'].astype(str)
df_1['JL_POCEntity']= df_1['JL_POCEntity'].astype(str)

In [ ]:
#Function to calculate similarity of Nouns, Pronouns and Entities

In [ ]:
def pronoun_similarity(df_1):
  if (df_1['JL_SitePronoun'] == "") | (df_1['JL_POCPronoun'] == "") :
    pass
  else:
    return nlp(df_1['JL_SitePronoun']).similarity(nlp(df_1['JL_POCPronoun']))

def noun_similarity(df_1):
    if (df_1['JL_SiteNoun'] == "") | (df_1['JL_POCNoun'] == ""):
      pass
    else:
      return nlp(df_1['JL_SiteNoun']).similarity(nlp(df_1['JL_POCNoun']))

def entity_similarity(df_1):
    if (df_1['JL_SiteEntity'] == "") | (df_1['JL_POCEntity'] == ""):
      pass
    else:
      return nlp(df_1['JL_SiteEntity']).similarity(nlp(df_1['JL_POCEntity']))

In [ ]:
df_1['Noun_Similarity'] = df_1.apply(noun_similarity,axis=1)

In [ ]:
df_1['Pronoun_Similarity'] = df_1.apply(pronoun_similarity,axis=1)

In [ ]:
df_1['Entity_Similarity'] = df_1.apply(entity_similarity,axis=1)

In [ ]:
df_1

In [ ]:
#latitude longitude for all PostCode in the UK to enable postCode comparison

In [ ]:
df_postcode = pd.read_csv(r'#readinspreadsheet with all UK postcodes including Long and Lat')

In [ ]:
df_postcode_JLsite = df_postcode[['Postcode','uk latitude','uk longitude']]
df_postcode_POC = df_postcode[['Postcode','uk latitude','uk longitude']]

In [ ]:
df_postcode_JLsite.columns = ['JL_PostCode', 'JLsite_latitude', 'JLsite_longitude']
df_postcode_POC.columns = ['POC_PostCode', 'POC_latitude', 'POC_longitude']

In [ ]:
df_1 = df_1.merge(df_postcode_JLsite,how='left',on='JL_PostCode')

In [ ]:
df_1 = df_1.merge(df_postcode_POC,how='left',on='POC_PostCode')
df_1

In [ ]:
df_1.fillna('0',inplace=True)

In [ ]:
import geopy.distance

coords_1 = (57.097358, -2.261098)
coords_2 = (57.097358, -3)

#test to calculate the distance between two coordinates
print(geopy.distance.geodesic(coords_1, coords_2).km)

In [ ]:
def distance(df_1):
  if (df_1['JLsite_latitude'] != 0) | (df_1['JLsite_longitude'] != 0) | (df_1['POC_latitude'] != 0) | (df_1['POC_longitude'] != 0):
    coords_1 = (df_1['JLsite_latitude'], df_1['JLsite_longitude'])
    coords_2 = (df_1['POC_latitude'], df_1['POC_longitude'])
    return geopy.distance.geodesic(coords_1, coords_2).km
  else:
    return np.nan

In [ ]:
df_1['Postcode Distance'] = df_1.apply(distance,axis=1)
df_1

In [ ]:
def total_score(df_1):

  if (df_1['Noun_Similarity'] != 0) & (df_1['Pronoun_Similarity'] != 0) & (df_1['Entity_Similarity'] != 0):
    return 3

  if (df_1['Noun_Similarity'] != 0) & ((df_1['Pronoun_Similarity'] != 0) | (df_1['Entity_Similarity'] != 0)):
    return 2

  if (df_1['Pronoun_Similarity'] != 0) & ((df_1['Noun_Similarity'] != 0) | (df_1['Entity_Similarity'] != 0)):
    return 2

  if (df_1['Entity_Similarity'] != 0) & ((df_1['Noun_Similarity'] != 0) | (df_1['Entity_Similarity'] != 0)):
    return 2


  if (df_1['Noun_Similarity'] != 0) & ((df_1['Pronoun_Similarity'] == 0) & (df_1['Entity_Similarity'] == 0)):
    return 1

  if (df_1['Pronoun_Similarity'] != 0) & ((df_1['Noun_Similarity'] == 0) & (df_1['Entity_Similarity'] == 0)):
    return 1

  if (df_1['Entity_Similarity'] != 0) & ((df_1['Noun_Similarity'] == 0) & (df_1['Entity_Similarity'] == 0)):
    return 1

  if (df_1['Entity_Similarity'] == 0) & (df_1['Noun_Similarity'] == 0) & (df_1['Entity_Similarity'] == 0):
    return 0

In [ ]:
df_1['Total Score'] = df_1.apply(total_score,axis=1)
df_1

In [ ]:
 def condition(df_1):

   if (df_1['JL_Site'] == df_1['POC_Site']) & (df_1['JL_PostCode'] == df_1['POC_PostCode'] ) & (df_1['JL_Address'] == df_1['POC_Address'] ):
     return "Match"

   if (df_1['JL_Address'] != df_1['POC_Address'] ):
     return "Mis-match Address"

   if (df_1['Postcode Distance'] == 0) & (df_1['JL_Site'] != df_1['POC_Site']):
     return "Mis-match Check Name"

   if (df_1['Postcode Distance'] > 0) & (df_1['JL_Site'] == df_1['POC_Site']) :
     return "Mis-match Check Postcode"    
  
   else:
     return "Mis-Match"

In [ ]:
df_1['Check'] = df_1.apply(condition,axis=1)

In [ ]:
df_1.to_excel('Finaltest.xlsx')